In [11]:
import function.functions as functions
# import function.functions_nocuda as functions # use this if you can't use cuda and the GPU

from qutip import *
import numpy as np

In [12]:
# Set device and parameters in this part

############################ Select device ##############################
# parameters of the device can be set in the parameters folder
device = 'charge_qubit' # change here to change the device: charge_qubit, flopping_spin, flopping_charge, transmon, fluxonium

data = np.load('data/params/'+device+'.npz', allow_pickle=True)
H_sys, drive_op, wq, g, kappa, num_A, dim = data['H_sys'], data['drive_op'], data['wq'], data['g'], data['kappa'], data['num_A'], data['dim']
###############################################################################

########################### Parameters of the paper ###########################
if device == 'charge_qubit':
    A_q = 0.05*wq
    ground, excited = 0, 1

    w_d = 0.3*wq # frequency at which the qubit is driven
                   # repeat the simulation for 0.3, 0.8, 0.9, 1.1, 1.15, 1.5 to obtain the data for Fig. 2a  of the main and Fig. S2a of the supplementary material

    w_d_disp = 0.9*wq # frequency at which the cavity is driven for dispersive readout
                        # in the paper, 0.9 for w_r < w_q and 1.1 for w_r > w_q

    N_rep =  15 # this means that we will have 2*N_rep+1 replicas
    N_fock = 15

    n_states = 2

    compensation = False # no need to compensate the charge qubit
    dispersive = True # dispersive readout

    proj = None

elif device == 'flopping_spin':
    A_q = 0.2*wq
    ground, excited = 0, 1
    w_d = 1.4*wq # frequency at which the qubit is driven  
    w_d_disp = 1.4*wq # frequency at which the cavity is driven for dispersive readout

    N_rep =  5 # this means that we will have 2*N_rep+1 replicas
    N_fock = 20

    n_states = 4

    compensation = True # both True and False in the paper
    dispersive = False # dispersive readout

    proj = None

elif device == 'transmon':
    A_q = 0.037*wq
    ground, excited = 0, 1
    w_d = 0.77*wq # frequency at which the qubit is driven
    w_d_disp = 0.77*wq # frequency at which the cavity is driven for dispersive readout

    N_rep = 5 # this means that we will have 2*N_rep+1 replicas
    N_fock = 11
    
    n_states = 8

    compensation = True
    dispersive = False # dispersive readout

    cutoff = 4
    proj = Qobj(np.diag(np.concatenate((np.zeros(cutoff),np.ones(n_states-cutoff)))))

elif device == 'fluxonium':
    A_q = 0.6*wq
    ground, excited = 0, 1
    w_d = 1.92*wq # frequency at which the qubit is driven
    w_d_disp = 1.92*wq # frequency at which the cavity is driven for dispersive readout

    N_rep = 5 # this means that we will have 2*N_rep+1 replicas
    N_fock = 11

    n_states = 8

    compensation = True
    dispersive = False # dispersive readout

    cutoff = 4
    proj = Qobj(np.diag(np.concatenate((np.zeros(cutoff),np.ones(n_states-cutoff)))))

qubit_state_list = [ground, excited]
tlist = np.linspace(0,8/kappa,num=500)

H_sys = Qobj(H_sys[0:n_states,0:n_states]) # truncation
drive_op = Qobj(drive_op[0:n_states,0:n_states]) # truncation
################################################################################

########################### Custom parameters #################################
# test other parameters A_q, ground, excited, w_r, w_r_disp, compensation

save_file = True # test mode or save data to generate the data of the figure of the paper
################################################################################

In [13]:
if save_file:
    fname = 'data/'+device+'/single_case/A_q='+str(np.round(A_q/wq,3))+'_w_d='+str(np.round(w_d/wq,3))\
        +'_w_d_disp='+str(np.round(w_d_disp/wq,3))+'_N_fock='+str(N_fock)+'_N_rep='+str(N_rep)+'_n_states='+str(n_states)\
            +'_dim='+str(dim)+'_compensation='+str(compensation)+'_dispersive='+str(dispersive)

    data = open(fname+'.txt', "w")
    data.close()

    data = open(fname+'_disp.txt', "w")
    data.close()

else:
    fname = None

############################# Longitudinal readout ##############################
A_list, dd_real, dd2_real, z0 = functions.get_derivatives(N_rep,A_q,H_sys,drive_op,[w_d],n_states,num_A,fname,True,qubit_state_list)

index_A = np.abs(A_list[0:(num_A-2)]-A_q).argmin()

message = 'derivatives of the spectrum computed in A_q/w_q='+str(A_list[index_A]/wq)
print(message)

if fname != None:
    with open(fname+'.txt', "a") as data:
        data.write(message+"\n")

g1 = g*dd_real[0,index_A,excited]
g0 = g*dd_real[0,index_A,ground]

chi1 = g**2*(dd2_real[0,index_A,excited]+1/A_q*dd_real[0,index_A,excited])
chi0 = g**2*(dd2_real[0,index_A,ground]+1/A_q*dd_real[0,index_A,ground])

g_parallel = 1/2*(g1-g0)
chi = 1/2*(chi1-chi0)

g_sum = 1/2*(g1+g0)
chi_sum = 1/2*(chi1+chi0)

if(compensation):
    w_r = w_d-chi_sum # shifted resonator frequency
    A_d = -2*g_sum # compensation tone
else:
    w_r = w_d
    A_d = 0

############################# Dispersive readout ##############################
A_list_disp, dd_real_disp, dd2_real_disp = functions.get_derivatives(N_rep,0.01*wq,H_sys,drive_op,[w_d_disp],n_states,num_A,fname+'_disp')

message = '(dispersive) derivatives of the spectrum computed in A_q/w_q='+str(A_list_disp[2]/wq)
print(message)

if fname != None:
    with open(fname+'_disp.txt', "a") as data:
        data.write(message+"\n")

chi_disp1 = g**2*dd2_real_disp[0,2,excited]
chi_disp0 = g**2*dd2_real_disp[0,2,ground]

chi_disp = chi_disp1-chi_disp0
chi_disp_sum = chi_disp1+chi_disp0

A_r = (g_parallel*kappa/chi_disp*(chi_disp**2+kappa**2/4)/(chi**2+kappa**2/4)) # match steady state SNR of longitudinal readout
phi = 3/2*np.pi # for dispersive readout

if(compensation):
    w_r_disp = (w_d_disp-chi_disp_sum) # shifted resonator frequency
else:
    w_r_disp = w_d_disp

derivatives of the spectrum computed in A_q/w_q=0.04983221476510068
(dispersive) derivatives of the spectrum computed in A_q/w_q=0.0001476510067114094


In [14]:
exp_a = [[],[]]
exp_a_an = [[],[]]

exp_a_disp = [[],[]]
exp_a_disp_an = [[],[]]

gamma = kappa*(g/(wq-w_d))**2 # Purcell
gamma_disp = kappa*(g/(wq-w_d_disp))**2 # Purcell for dispersive readout

exp_a, exp_fock, exp_proj = functions.real_time_dynamics(H_sys,A_q,np.array([A_d]),[w_r],[w_d],0,g,drive_op,n_states,kappa,qubit_state_list,tlist,N_fock,proj,fname)
exp_a = [exp_a[0,0], exp_a[0,1]]

if dispersive:
    exp_a_disp, exp_fock_disp, exp_proj_disp = functions.real_time_dynamics(H_sys,0,np.array([A_r]),[w_r_disp],[w_d_disp],phi,g,drive_op,n_states,kappa,qubit_state_list,tlist,N_fock,proj,fname+'_disp')
    exp_a_disp = [exp_a_disp[0,0], exp_a_disp[0,1]]

for idx,qubit_state in enumerate(qubit_state_list):
    
    res_an = functions.analytical_time_dynamics(z0[0,0,idx],w_r,w_d,A_d,0,g_parallel,g_sum,chi,chi_sum,kappa,gamma,tlist)
    exp_a_an[idx] = res_an

    if dispersive:

        if qubit_state == ground:
            z0_disp = -1
        else:
            z0_disp = 1
        
        res_disp_an = functions.analytical_time_dynamics(z0_disp,w_r_disp,w_d_disp,A_r,phi,0,0,chi_disp,chi_disp_sum,kappa,gamma_disp,tlist)
        exp_a_disp_an[idx] = res_disp_an

In [ ]:
if save_file:
    if dispersive:
        np.savez(fname, qubit_state_list=qubit_state_list, \
                exp_a=exp_a, exp_a_disp=exp_a_disp, exp_a_an=exp_a_an, exp_a_disp_an=exp_a_disp_an,\
                    tlist=tlist, A_q=A_q, g=g, kappa=kappa, w=w_r, wq=wq, g_parallel=g_parallel, exp_fock=exp_fock, exp_proj=exp_proj,\
                        exp_fock_disp=exp_fock_disp, exp_proj_disp=exp_proj_disp, A_list=A_list)
    else:
        np.savez(fname, qubit_state_list=qubit_state_list, \
        exp_a=exp_a, exp_a_disp=exp_a_disp, exp_a_an=exp_a_an, exp_a_disp_an=exp_a_disp_an,\
            tlist=tlist, A_q=A_q, g=g, kappa=kappa, w=w_r, wq=wq, g_parallel=g_parallel, exp_fock=exp_fock, exp_proj=exp_proj, A_list=A_list)